## Example 01 - Basic Hopper Operation

**Example description:** Example of a trailing suction hopper dredge shipping sediment from origin to destination site.

* [**0. Import required libraries:**](#0.-Import-required-libraries)<br>
* [**1. Define work method:**](#1.-Define-work-method)<br>
   * [**1.1 Sites:**](#1.1-Define-the-project-sites)<br>
   * [**1.2 Equipment:**](#1.2-Define-the-project-equipment)<br>
   * [**1.3 Activities:**](#1.3-Define-the-activity)<br>
* [**2. Run the simulation:**](#2.-Run-the-simulation)<br>
* [**3. Post processing:**](#3.-Postprocessing)<br>

## 0. Import required libraries

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import numpy as np

# digital twin package
import digital_twin.core as core
import digital_twin.model as model
import digital_twin.plot as plot

# Additional import to save the initialization of the simulation
import digital_twin.savesim as savesim

In [2]:
# Create simulation environment
simulation_start = datetime.datetime.now()

my_env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
my_env.epoch = time.mktime(simulation_start.timetuple())

## 1. Define work method

### 1.1 Define the project sites
You can specify a project site object by entering mix-ins:

    core.Identifiable - enables you to give the object a name
    core.Log - enables you to log all discrete events in which the object is involved
    core.Locatable - enables you to add coordinates to extract distance information and visualize
    core.HasContainer - enables you to add information on the material available at the site
    core.HasResource - enables you to add information on serving equipment
    core.HasWeather - enables you to add weather conditions

#### First create objects with the desired properties

In [3]:
# The generic site class
Site = type('Site', (core.Identifiable, # Give it a name
                     core.Log,          # Allow logging of all discrete events
                     core.Locatable,    # Add coordinates to extract distance information and visualize
                     core.HasContainer, # Add information on the material available at the site
                     core.HasResource), # Add information on serving equipment
            {})                         # The dictionary is empty because the site type is generic

#### Next specify the properties for sites you wish to create

In [4]:
# Information on the extraction site - the "from site" - the "win locatie"
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)  # lon, lat

data_from_site = {"env": my_env,                                # The simpy environment defined in the first cel
                  "name": "Winlocatie",                         # The name of the site
                  "ID": "6dbbbdf4-4589-11e9-a501-b469212bff5b", # For logging purposes
                  "geometry": location_from_site,               # The coordinates of the project site
                  "capacity": 500_000,                          # The capacity of the site
                  "level": 500_000}                             # The actual volume of the site

In [5]:
# Information on the dumping site - the "to site" - the "dump locatie"
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)     # lon, lat

data_to_site = {"env": my_env,                                # The simpy environment defined in the first cel
                "name": "Dumplocatie",                        # The name of the site
                "ID": "6dbbbdf5-4589-11e9-82b2-b469212bff5b", # For logging purposes
                "geometry": location_to_site,                 # The coordinates of the project site
                "capacity": 500_000,                          # The capacity of the site
                "level": 0}                                   # The actual volume of the site (empty of course)

#### Finally create specific instances of the predefined objects with the specified properties

In [6]:
# The two objects used for the simulation
from_site = Site(**data_from_site)
to_site   = Site(**data_to_site)

### 1.2 Define the project equipment
You can specify a vessel object by entering mix-ins:

    core.Identifiable - enables you to give the object a name
    core.Log - enables you to log all discrete events in which the object is involved
    core.ContainerDependentMovable - A moving container, so capacity and location
    core.Processor - Allow for loading and unloading
    core.HasResource - Add information on serving equipment
    core.HasDepthRestriction - Add information on depth restriction 

#### First create objects with the desired properties

In [7]:
# The generic class for an object that can move and transport (a TSHD for example)
TransportProcessingResource = type('TransportProcessingResource', 
                                   (core.Identifiable,              # Give it a name
                                    core.Log,                       # Allow logging of all discrete events
                                    core.ContainerDependentMovable, # A moving container, so capacity and location
                                    core.Processor,                 # Allow for loading and unloading
                                    core.HasResource,               # Add information on serving equipment
                                    ),                 # Initialize spill terms
                                   {})

#### Next specify the properties for vessel(s) you wish to create

In [8]:
# For more realistic simulation you might want to have speed dependent on the volume carried by the vessel
def compute_v_provider(v_empty, v_full):
    return lambda x: x * (v_full - v_empty) + v_empty

def compute_loading(rate):
    return lambda current_level, desired_level: (desired_level - current_level) / rate

def compute_unloading(rate):
    return lambda current_level, desired_level: (current_level - desired_level) / rate

In [9]:
# TSHD variables
data_hopper = {"env": my_env,                                       # The simpy environment 
               "name": "Hopper 01",                                 # Name
               "ID": "6dbbbdf6-4589-11e9-95a2-b469212bff5b",        # For logging purposes
               "geometry": location_from_site,                      # It starts at the "from site"
               "loading_func": compute_loading(1.5),                # Loading rate
               "unloading_func": compute_unloading(1.5),            # Unloading rate
               "capacity": 5_000,                                   # Capacity of the hopper - "Beunvolume"
               "compute_v": compute_v_provider(5, 4.5)}             # Variable speed 

#### Finally create specific instances of the predefined objects with the specified properties

In [10]:
# The simulation object
hopper = TransportProcessingResource(**data_hopper)

### 1.3 Define the activity

In [11]:
# Create activity
activity = model.Activity(env = my_env,           # The simpy environment defined in the first cel
                          name = "Soil movement", # We are moving soil
                          ID = "6dbbbdf7-4589-11e9-bf3b-b469212bff5b", # For logging purposes
                          origin = from_site,     # We originate from the from_site
                          destination = to_site,  # And therefore travel to the to_site
                          loader = hopper,        # The benefit of a TSHD, all steps can be done
                          mover = hopper,         # The benefit of a TSHD, all steps can be done
                          unloader = hopper,      # The benefit of a TSHD, all steps can be done
                          start_condition = None) # We can start right away and do not stop

### 2. Run the simulation

In [12]:
my_env.run()

print("\n*** Installation of dike finished in {} ***".format(datetime.timedelta(seconds=int(my_env.now - my_env.epoch))))


Time = 2019-03-14 09:33
Start condition is satisfied, Soil movement transporting from Winlocatie to Dumplocatie started.

Time = 2019-03-26 16:47
Stop condition is satisfied, Soil movement transporting from Winlocatie to Dumplocatie completed.

*** Installation of dike finished in 12 days, 7:14:22 ***


### 3. Postprocessing

#### Vessel planning

In [13]:
vessels = [hopper]

activities = ['loading', 'unloading', 'sailing filled', 'sailing empty']
colors = {0:'rgb(55,126,184)', 1:'rgb(255,150,0)', 2:'rgb(98, 192, 122)', 3:'rgb(98, 141, 122)'}

plot.vessel_planning(vessels, activities, colors)

### Save all logs

In [14]:
sites = [from_site, to_site]
equipment = [hopper]
activities = [activity]

save = savesim.LogSaver(sites, equipment, activities, 
                        simulation_id = '5e1d7c36-458e-11e9-816b-b469212bff5b', simulation_name = "Example 01",
                        location = "Simulation Results", overwrite = True)